In [15]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from imblearn.over_sampling import SMOTE
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, LSTM
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import roc_auc_score, precision_score
from sklearn import metrics 
import numpy as np
from itertools import chain

In [3]:
#load train, validation, and test data
train = pd.read_csv("/Users/jcheah/Documents/MITB/Term 3/AML/Final Project/FDIC_train_new")
val = pd.read_csv("/Users/jcheah/Documents/MITB/Term 3/AML/Final Project/FDIC_val_new")
test = pd.read_csv("/Users/jcheah/Documents/MITB/Term 3/AML/Final Project/FDIC_test_new")

In [4]:
#drop unnecessary column
train.drop(['Unnamed: 0'], axis = 1, inplace = True)
val.drop(['Unnamed: 0'], axis = 1, inplace = True)
test.drop(['Unnamed: 0'], axis = 1, inplace = True)

In [5]:
train.head()

,total_assets_1,log_TA_1,NI_to_TA_1,Equity_to_TA_1,NPL_to_TL_1,REO_to_TA_1,ALLL_to_TL_1,core_deposits_to_TA_1,brokered_deposits_to_TA_1,liquid_assets_to_TA_1,...,assets_growth_4,term_spread_4,stock_mkt_growth_4,real_gdp_growth_4,unemployment_rate_change_4,treasury_yield_3m_4,bbb_spread_4,bbb_spread_change_4,max_total_assets_4,Failure
0,11300.0,-1.915942,0.234778,-0.577477,-0.070827,-0.374868,-0.335405,-0.263400,-0.182975,-1.224314,...,-0.704106,1.268158,-1.707009,-0.119978,0.202649,0.066902,0.216947,0.161157,43258.0,0
1,11206.0,-1.922102,0.489899,-0.550654,-0.362882,-0.234268,-0.376707,-0.514923,-0.182975,-1.326412,...,-0.142293,1.268158,-1.707009,-0.119978,0.202649,0.066902,0.216947,0.161157,43258.0,0
2,12234.0,-1.857377,0.232075,-0.616682,-0.658730,-0.223355,-0.431984,-0.617907,-0.182975,-1.262651,...,1.089193,1.268158,-1.707009,-0.119978,0.202649,0.066902,0.216947,0.161157,43258.0,0
3,11781.0,-1.885201,0.389738,-0.581886,-0.695820,-0.374868,-0.342970,-0.214020,-0.182975,-1.300326,...,-0.187529,0.526578,-2.295988,-0.253927,-0.454312,0.006692,1.029487,1.698409,43258.0,0
4,11853.0,-1.880708,0.547953,-0.545033,0.499780,-0.374868,-0.286405,-0.051837,-0.182975,-1.177203,...,0.633063,0.433881,1.674053,0.951609,0.531129,-0.354567,-0.189323,-0.863677,43258.0,0


In [6]:
train.shape

(8838, 89)

In [7]:
y_train = train.iloc[:, -1]
x_train = train.iloc[:, :88]
#oversample the positive classes by SMOTE
x_train, y_train = SMOTE().fit_resample(x_train, y_train)
#normalise the data
x_train = MinMaxScaler().fit_transform(x_train) 
y_train = y_train.astype('int')
#reshape the data to 3 dimensions by num of rows, num of quarters and num of columns
x_train = x_train.reshape(x_train.shape[0], 4, 22)


In [8]:
y_val = val.iloc[:, -1]
x_val = val.iloc[:, :88]
x_val = MinMaxScaler().fit_transform(x_val) 
y_val = y_val.astype('int')
x_val = x_val.reshape(x_val.shape[0], 4, 22)

In [9]:
y_test = test.iloc[:, -1]
x_test = test.iloc[:, :88]
x_test = MinMaxScaler().fit_transform(x_test) 
y_test = y_test.astype('int')
x_test = x_test.reshape(x_test.shape[0], 4, 22)

In [10]:
pd.DataFrame(y_train).Failure.value_counts()

1    8512
0    8512
Name: Failure, dtype: int64

In [11]:
pd.DataFrame(y_val).Failure.value_counts()

0    2091
1      66
Name: Failure, dtype: int64

In [12]:
pd.DataFrame(y_test).Failure.value_counts()

0    2708
1      70
Name: Failure, dtype: int64

In [165]:
def buildManyToOneModel(shape):
    model = Sequential()
    model.add(LSTM(128, input_shape=(shape[1], shape[2]), return_sequences = True))
    model.add(Dropout(0.3))
    model.add(LSTM(64, return_sequences = True, activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(LSTM(32, return_sequences = True, activation = 'relu'))
    model.add(Dropout(0.3))
    model.add(LSTM(16, activation = 'relu'))
    
    model.add(Dense(16, activation = 'relu'))
    # output shape: (1, 1)
    model.add(Dense(1, activation = 'relu'))
              
    opt = tf.keras.optimizers.Adam(lr = 1e-2)
    model.compile(loss="mse", optimizer=opt)
    model.summary()
    return model

model = buildManyToOneModel(x_train.shape)
model.fit(x_train, y_train, epochs=4, batch_size=64, validation_data=(x_val, y_val))

Model: "sequential_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_122 (LSTM)              (None, 4, 128)            77312     
_________________________________________________________________
dropout_85 (Dropout)         (None, 4, 128)            0         
_________________________________________________________________
lstm_123 (LSTM)              (None, 4, 64)             49408     
_________________________________________________________________
dropout_86 (Dropout)         (None, 4, 64)             0         
_________________________________________________________________
lstm_124 (LSTM)              (None, 4, 32)             12416     
_________________________________________________________________
dropout_87 (Dropout)         (None, 4, 32)             0         
_________________________________________________________________
lstm_125 (LSTM)              (None, 16)              

In [166]:
lstm_pred = model.predict(x_val)
lstm_pred = np.ndarray.tolist(lstm_pred)
#remove the list 
lstm_pred = list(chain.from_iterable(lstm_pred))
len(lstm_pred)

2157

In [167]:
threshold = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for th in threshold:
    lstm_pred_int = [1 if i >= th else 0 for i in lstm_pred]
    precision = metrics.precision_score(y_val, lstm_pred_int)
    recall = metrics.recall_score(y_val, lstm_pred_int)
    F2_score = metrics.fbeta_score(y_val, lstm_pred_int, beta = 2)
    area_under_roc = roc_auc_score(y_val, lstm_pred)

    print(f'\n******** For i = {th} ****** \n',
          f'Precision: {precision} \n',
          f'Recall: {recall} \n',
          f'F2_score: {F2_score} \n',
          f'ROC_AUC_Score: {area_under_roc}')
    lstm_pred_int = []


******** For i = 0.1 ****** 
 Precision: 0.08672798948751642 
 Recall: 1.0 
 F2_score: 0.32195121951219513 
 ROC_AUC_Score: 0.9394664000115936

******** For i = 0.2 ****** 
 Precision: 0.09065934065934066 
 Recall: 1.0 
 F2_score: 0.3326612903225806 
 ROC_AUC_Score: 0.9394664000115936

******** For i = 0.3 ****** 
 Precision: 0.09361702127659574 
 Recall: 1.0 
 F2_score: 0.34055727554179566 
 ROC_AUC_Score: 0.9394664000115936

******** For i = 0.4 ****** 
 Precision: 0.09850746268656717 
 Recall: 1.0 
 F2_score: 0.3533190578158458 
 ROC_AUC_Score: 0.9394664000115936

******** For i = 0.5 ****** 
 Precision: 0.10107197549770292 
 Recall: 1.0 
 F2_score: 0.3598691384950927 
 ROC_AUC_Score: 0.9394664000115936

******** For i = 0.6 ****** 
 Precision: 0.10610932475884244 
 Recall: 1.0 
 F2_score: 0.3724604966139955 
 ROC_AUC_Score: 0.9394664000115936

******** For i = 0.7 ****** 
 Precision: 0.11478260869565217 
 Recall: 1.0 
 F2_score: 0.3933253873659118 
 ROC_AUC_Score: 0.93946640001159

/Users/jcheah/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [170]:
model.save('bank_bankruptcy_lstm_model.h5')

In [171]:
model = load_model('bank_bankruptcy_lstm_model.h5')

In [168]:
#predict on test_data
y_pred  = model.predict(x_test.reshape(x_test.shape[0], 4, 22))

In [169]:
#test_data by different thresholds

threshold = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

for th in threshold:
    
    y_class = [1 if i >= th else 0 for i in y_pred] 
    precision = metrics.precision_score(y_test, y_class)
    recall = metrics.recall_score(y_test, y_class)
    F2_score = metrics.fbeta_score(y_test, y_class, beta = 2)
    area_under_roc = roc_auc_score(y_test, y_pred)

    print(f'\n******** For i = {th} ****** \n',
          f'Precision: {precision} \n',
          f'Recall: {recall} \n',
          f'F2_score: {F2_score} \n',
          f'ROC_AUC_Score: {area_under_roc}')


******** For i = 0.1 ****** 
 Precision: 0.05546218487394958 
 Recall: 0.9428571428571428 
 F2_score: 0.22448979591836737 
 ROC_AUC_Score: 0.8811932897235705

******** For i = 0.2 ****** 
 Precision: 0.05871886120996441 
 Recall: 0.9428571428571428 
 F2_score: 0.23504273504273507 
 ROC_AUC_Score: 0.8811932897235705

******** For i = 0.3 ****** 
 Precision: 0.060129509713228495 
 Recall: 0.9285714285714286 
 F2_score: 0.2387950036737693 
 ROC_AUC_Score: 0.8811932897235705

******** For i = 0.4 ****** 
 Precision: 0.06177606177606178 
 Recall: 0.9142857142857143 
 F2_score: 0.243161094224924 
 ROC_AUC_Score: 0.8811932897235705

******** For i = 0.5 ****** 
 Precision: 0.06451612903225806 
 Recall: 0.9142857142857143 
 F2_score: 0.25157232704402516 
 ROC_AUC_Score: 0.8811932897235705

******** For i = 0.6 ****** 
 Precision: 0.06823027718550106 
 Recall: 0.9142857142857143 
 F2_score: 0.2627257799671593 
 ROC_AUC_Score: 0.8811932897235705

******** For i = 0.7 ****** 
 Precision: 0.07655

/Users/jcheah/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
